In [1]:
from random import seed
from random import randrange
from csv import reader
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
def load_csv(filename):
    file = open(filename, "rt")
    lines = reader(file)
    dataset = list(lines)
    return dataset

In [3]:
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

In [4]:
# Функция разделения набора данных на k подвыборок:
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [5]:
# Функция подсчета значения метрики "accuracy" в %
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [6]:
# Функция оценки точности работы алгоритма с помощью кросс-валидации
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

In [7]:
# Функуия разделения выборки на две подвыборки в узле дерева
# на основе наименования атрибута и его значения
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

In [8]:
# Функция вычисления индекса Джини для разделенной выборки
def gini_index(groups, classes):
    # подсчет общего количества элементов в точке разделения
    n_instances = float(sum([len(group) for group in groups]))
    # суммирование взвещенного индекса Джини для каждой группы
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # проверка деления на 0
        if size == 0:
            continue
        score = 0.0
       # получение оценки для группы на основе оценки для каждого из классов
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # взвешивание оценок групп на основе их относительного размера
        gini += (1.0 - score) * (size / n_instances)
    return gini

In [9]:
# Функция выбора наилучшей точки разделения для набора данных
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

In [10]:
# Функция определения значения для листового узла дерева
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

In [11]:
# Функция создания узла разделения или получения значения для листового узла
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # Проверка на отсутствие разделения
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # Проверка на максимальную глубину дерева
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # Обработка левой ветви
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # Обработка правой ветви
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

In [12]:
# Функция построения дерева решений
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

In [13]:
# Функция печати дерева решений
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

In [14]:
# Функция выполнения предсказания с помощью дерева решений
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [15]:
# Алгоритм дерева решений
def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return(predictions)

Тестирование дерева решений на наборе данных об ирисах

In [16]:
# Загрузка и подготовка данных
data = pd.read_csv('iris.data', header = None)
data.columns = ['sepal_length','sepal_width', 'petal_length', 'petal_width', 'class']
data.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [17]:
reg_exp = r'(\w+)-(\w+)'
new_columns = data['class'].str.extract(reg_exp)
new_columns.columns = ['iris', 'iris_class']

In [18]:
data = data.join(new_columns)

In [19]:
label_enc = LabelEncoder()

In [20]:
label_iris_class = label_enc.fit_transform(data['iris_class'])
label_iris_class = pd.DataFrame(label_iris_class, columns = ['iris_class'])

In [21]:
data.drop(['iris', 'class', 'iris_class'], axis = 1, inplace = True)

In [22]:
data = data.join(label_iris_class)

In [23]:
data.head()

,sepal_length,sepal_width,petal_length,petal_width,iris_class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [24]:
data.tail()

,sepal_length,sepal_width,petal_length,petal_width,iris_class
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2
149,5.9,3.0,5.1,1.8,2


In [25]:
# Преобразование данных из Dataframe в список списков
iris_dataset = data.values.tolist()

In [27]:
# Оценка алгоритма
seed(100)
n_folds = 5
max_depth = 5
min_size = 10
scores = evaluate_algorithm(iris_dataset, decision_tree, n_folds, max_depth, min_size)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores) / float(len(scores))))

Scores: [90.0, 96.66666666666667, 93.33333333333333, 93.33333333333333, 96.66666666666667]
Mean Accuracy: 94.000%
